<a href="https://colab.research.google.com/github/Saif-Shines/pk-cookbook/blob/autogen-portkey-klyst-170/product/observability_and_resilience_essentials_for_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Observability and Resilience Essentials for Autogen

Autogen lets you bring collaborative conversations among LLMs, Humans, and Tools to reason together and take action. All of those Agents can be remarkably composed together to be the powerful components of your LLM apps.

On the other hand, Portkey integrates production-grade monitoring and resiliency capabilities into the Autogen workflows so you have all the insights you need to put your LLM based apps in produciton.

Some include:

1. Calling 100+ LLMs (open & closed)
2. Logging & analyzing LLM usage
3. Caching responses
4. Automating fallbacks, retries, timeouts, and load balancing

Let's see how you can leverage Autogen and Portkey.


## Setup

Install the client sdk for Portkey and Autogen using the following command

In [ ]:
!pip install pyautogen

In [ ]:
!pip install portkey-ai

Once the SDKs are installed, you need authentication keys to instantiate them.

1. Login to the [Portkey app](https://www.portkey.ai/).
2. To integrate OpenAI with Portkey, add your OpenAI API key as a [Virtual Key](https://portkey.ai/docs/product/ai-gateway-streamline-llm-integrations/virtual-keys).
3. This will give you a disposable key that you can use and rotate instead of directly using the OpenAI API key.

## Integrate Autogen with Portkey

Autogen lets you define a <code>[config_list](https://microsoft.github.io/autogen/docs/topics/llm_configuration#introduction-to-config_list)</code> in which Portkey can be listed seamlessly.


In [ ]:
import autogen

In [ ]:
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

In [ ]:
config_list = [
    {
        "api_key": '<ignored>',
        "model": "gpt-4",
        "max_tokens": 512,
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = "PORTKEY_API_KEY",
            virtual_key="OPENAI_VIRTUAL_KEY",
            trace_id="tracing_autogen",
            metadata={"_user": "USER_ID"}
        )
    }
]

The top-level `api_key` parameter should be the API key of the LLM provider, which in this case is OpenAI. However, since we changed the `base_url` to be Portkey’s AI gateway, the `virtual_key` takes precedence, and the top-level `api_key` is not respected.

The parameters under `default_headers` are sent to Portkey. Hence, `api_key` should be Portkey’s API key, and `trace_id` can be any identifier that helps you filter and view the request logs in the Portkey app.


## Task: Write Python code and store it in a file

Let’s create two Agents using Autogen — LLM Agent (Assistant Agent) and Human Agent (User Proxy Agent).

In [ ]:
llm_config={
    "config_list": config_list,
}

In [ ]:
assistant = autogen.AssistantAgent(
    name="Staff Engineer",
    llm_config=llm_config,
    system_message="You are very much experienced Python Programmer"
)

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

We defined the role of an `assistant` as an experienced staff engineer. Meanwhile, `user_proxy` participates in the conversation on your behalf. It replies less than 10 times without needing your input to do the task. To learn more about parameters in detail, visit [Autogen documentation](https://microsoft.github.io/autogen/docs/Getting-Started/).

Initiate the conversation to execute the task (s):

In [ ]:
task = """
Write python code to output numbers 1 to 10, and then store the code in a file
"""

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message=task
)

# See the root directory of this file to find the python file generated.

In [ ]:
task2 = """
Change the code in the file you just created to instead output numbers 1 to 200
"""

user_proxy.initiate_chat(
    assistant,
    message=task2
)

According to the task — The Python file will be created and updated programmatically in your current working directory.


## Switch from OpenAI to Anyscale

Porkey’s AI gateway (also [open source](https://github.com/portkey-ai/gateway)) can route your requests to 100+ LLMs using the OpenAI signature you already know. It can let you switch from GPT4 to Mistral by updating just two lines of code.


```js
config_list = [
    {
        "api_key": '<ignored>',
        "model": "mistralai/Mistral-7B-Instruct-v0.1",
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = "xxxxx",
            virtual_key="anyscxxxxx65",
            trace_id="tracing_autogen"
        )
    }
]
```


The same will apply to all the other providers like Azure, Mistral, Anyscale, Together, and more. See the [integration guides](https://portkey.ai/docs/welcome/integration-guides)!

Note that not all the models have capabilities (for example,  function calling) that enable Autogen to be drop-in compatible to accomplish the tasks.


## Trace and View Autogen Conversations

Portkey lets you inspect every interaction with an LLM on a single control panel in its app. You can surf through the logs, analytics, traces, and much more to get complete insights about your LLM app.

Autogen initiates the conversation and gets the task done. With Portkey, you can see tokens used, costs, and inspect all the requests and responses to better optimize your apps.

![](https://raw.githubusercontent.com/Portkey-AI/portkey-cookbook/a4ebb6c7cb75f23be01d398cbbbe8fbe669a053a/integrations/images/observability-and-resilience-essentials-for-autogen/1-observability-and-resilience-essentials-for-autogen.gif)

## Segment the requests using metadata

You can also pass any `{"key":"value"}` pairs to segment the requests based on the metadata and get granular insights.


```js
config_list = [
    {
        "api_key": '<ignored>',
        "model": "mistralai/Mistral-7B-Instruct-v0.1",
        "max_tokens": 512,
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = "xxxxx",
            virtual_key="anyscale-xxxxx",
            trace_id="tracing_autogen",
            metadata={"_user": "USER_ID"}
        )
    }
]
```


Learn more about [tracing](https://portkey.ai/docs/product/observability-modern-monitoring-for-llms/traces) and [feedback](https://portkey.ai/docs/product/observability-modern-monitoring-for-llms/feedback).


## Automatically Handle LLM Failures

Portkey can handle temporary LLM downtimes and rate limit errors by applying fallbacks and loadbalancing the requests. To enable this, login to the Portkey app and **Configs** and click **Create**.


### 4XX and 5XX errors

For example, for setting up a fallback from OpenAI to Anthropic, the Gateway Config would be:


```json
{
    "strategy": { "mode": "fallback" },
    "targets": [
        { "virtual_key": "openai-virtual-key" },
        { "virtual_key": "anthropic-virtual-key" }
    ]
}
```

You can save this Config and get an associated Config ID to pass it as part of `default_headers`

```js
config_list = [
    {
        "api_key": '<ignored>',
        "model": "mistralai/Mistral-7B-Instruct-v0.1",
        "base_url": PORTKEY_GATEWAY_URL,
        "api_type": "openai",
        "default_headers": createHeaders(
            api_key = "xxxxx",
            virtual_key="annyscale-xxxxx",
            trace_id="tracing_autogen",
            config="CONFIG_ID"
          )
    }
]
```

### Handle Ratelimits

Similarly, you can loadbalance the requests against multiple LLMs or accounts to prevent any one account from hitting rate limit thresholds.

For example, to route your requests between 1 OpenAI and 2 Azure OpenAI accounts:

```json
{
    "strategy": { "mode": "loadbalance" },
    "targets": [
        { "virtual_key": "openai-virtual-key", "weight":1 },
        { "virtual_key": "azure-virtual-key-1", "weight":1 },
        { "virtual_key": "azure-virtual-key-2", "weight":1 }
    ]
}
```


Learn more such features as semantic caching, automatic retries and request timeout on the [documentation](https://portkey.ai/docs/product/ai-gateway-streamline-llm-integrations).


## Conclusion

Autogen combined with Portkey, will give your apps the collaborative power of LLMs and production-grade resiliency. If you have any questions or issues, reach out to us on [Discord](https://portkey.ai/community). On Discord, you will also meet many other practitioners who are putting their LLM apps into production.

